In [12]:
prefix = "../backend/app/data/"
import os, sys, json, time
sys.path.append(os.path.join(prefix,"model/UnifiedSKG"))
import warnings
warnings.filterwarnings("ignore")
import torch
from transformers import AutoTokenizer

from utils.configue import Configure
from models.unified.prefixtuning import Model

from filelock import FileLock
import nltk
with FileLock(".lock") as lock:
    nltk.download("punkt", quiet=True)
    nltk.download("stopwords", quiet=True)

In [13]:
def play(txt, model, tokenizer):
  print("=====❓Request=====")
  print(txt)
  tokenized_txt = tokenizer([txt], max_length=1024, padding="max_length", truncation=True)
  pred = tokenizer.batch_decode(
      model.generate(
        torch.LongTensor(tokenized_txt.data['input_ids']),
        torch.LongTensor(tokenized_txt.data['attention_mask']),
        num_beams=1, 
        max_length=256
        ), 
      skip_special_tokens=True 
  ) # More details see utils/dataset.py and utils/trainer.py
  print("=====💡Answer=====")
  print(pred)

In [15]:
# load model
tokenizer = AutoTokenizer.from_pretrained("hkunlp/from_all_T5_base_prefix_sql2text2", use_fast=False)
model_config_path = os.path.abspath(os.path.join(prefix,"model/UnifiedSKG/configure/Salesforce/T5_base_prefix_sql2text.cfg"))
args = Configure.Get('/data2/xingbo/chi2022/UnifiedSKG/configure/Salesforce/T5_base_prefix_sql2text.cfg')
model = Model(args)
model.load("hkunlp/from_all_T5_base_prefix_sql2text2")

prefix-tuning sequence length is 10.


In [17]:
struct_in = ""
text_in = "SELECT name ,  country ,  age FROM singer ORDER BY age DESC"
play("{} ; structed knowledge: {}".format(text_in, struct_in), model, tokenizer)

=====❓Request=====
SELECT name ,  country ,  age FROM singer ORDER BY age DESC ; structed knowledge: 
=====💡Answer=====
['What are the names, countries, and ages of all singers, ordered by age descending?']
